In [1]:
import re
import gc
import pickle
import argparse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import *
from keras import optimizers
from keras import backend as K
from collections import Counter
from keras.utils import np_utils
from gensim.models import word2vec
from sklearn.metrics import f1_score
from keras.regularizers import l1, l2
from keras.engine import InputSpec, Layer
from keras.models import Model, load_model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.layers import Input, Embedding, Dense
from sklearn.preprocessing import StandardScaler
from keras import optimizers, losses, activations
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
from keras.layers.noise import GaussianNoise
from models_def import Attention
config = argparse.Namespace()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
der=True
if der:
    train_dir = '../input/new_data/train_set.csv'
    test_dir = '../input/new_data/test_set.csv'
else:
    train_dir = '../input/train2.csv'
    test_dir = '../input/test2.csv'
train = pd.read_csv(train_dir)
test = pd.read_csv(test_dir)

In [3]:

def clear_text(df, num, col):
    word_seg = df[col]
    word_seg_list = word_seg.apply(lambda x: x.split(" "))

    word_counts = Counter()
    for line in word_seg_list:
        word_counts.update(line)

    counter_list = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    label = list(map(lambda x: x[0], counter_list[:num]))
    from tqdm import tqdm
    for i in tqdm(label):
        for j in word_seg_list:
            if i in j:
                j.remove(i)
    df[col] = word_seg_list
    df[col] = df[col].apply(lambda x: ' '.join(x))

In [4]:
all_data = pd.concat([train, test])
all_data = all_data.reset_index(drop=True)
train_offset = train.shape[0]

In [5]:
all_data['word_len'] = all_data['word_seg'].apply(len)
all_data['word_unique'] = all_data['word_seg'].apply(lambda comment: len(set( w for w in comment.split())))
all_data['word_unique_vs_len'] = all_data['word_unique'] /  all_data['word_len']

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [7]:
train = all_data.iloc[:train_offset,:]
test = all_data.iloc[train_offset:,:]

In [8]:
from sklearn.preprocessing import StandardScaler
features = train['word_unique_vs_len'].fillna(0)
test_features = test['word_unique_vs_len'].fillna(0)
features = features.reshape(-1, 1)
test_features = test_features.reshape(-1, 1)
ss = StandardScaler()
ss.fit(np.vstack((features, test_features)))
features = ss.transform(features)
test_features = ss.transform(test_features)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [9]:
df_y_train = (train["class"]-1).astype(int)
# column = "word_seg"
test_id = test[["id"]].copy()

In [10]:
all_data['text'] = all_data['word_seg'].str.cat([all_data.article], sep=' ',na_rep='')
all_data.drop(['word_seg', 'article'], axis = 1, inplace=True)

In [11]:
train = all_data.iloc[:train_offset,:]
test = all_data.iloc[train_offset:,:]

In [12]:
column = "text"
config.len_desc = 500000
tknzr_word = Tokenizer(num_words=config.len_desc)
tknzr_word.fit_on_texts(all_data[column].values)

In [13]:
count_thres = 4
low_count_words = [w for w, c in tknzr_word.word_counts.items() if c < count_thres]
# print(len(tknzr_word.texts_to_sequences(all_data[column].values)))
for w in tqdm(low_count_words):
    del tknzr_word.word_index[w]
    del tknzr_word.word_docs[w]
    del tknzr_word.word_counts[w]

100%|██████████| 849060/849060 [00:01<00:00, 747505.21it/s]


In [14]:
tr_word_seq = tknzr_word.texts_to_sequences(train[column].values)
te_word_seq = tknzr_word.texts_to_sequences(test[column].values)
config.maxlen = 800
tr_word_pad = pad_sequences(tr_word_seq, maxlen=config.maxlen)
te_word_pad = pad_sequences(te_word_seq, maxlen=config.maxlen)

In [15]:
from gensim.models import word2vec
max_features = 500000
vec_len = 600
EMBEDDING = '../feature/word2vec_file/word_and_article600d.w2v'
model = word2vec.Word2Vec.load(EMBEDDING)
word_index = tknzr_word.word_index
nb_words_desc = min(max_features, len(word_index))
embedding_matrix_desc = np.zeros((nb_words_desc, vec_len))
for word, i in word_index.items():
    if i >= max_features: continue
    try:
        embedding_vector = model[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix_desc[i] = embedding_vector

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [16]:
Y = df_y_train

In [17]:
from keras.engine import InputSpec, Layer
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [18]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.engine import Layer, InputSpec
from keras.layers import Flatten
import tensorflow as tf

class KMaxPooling(Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension).
    TensorFlow backend.
    """
    def __init__(self, k=1, **kwargs):
        super().__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)
        self.k = k

    def compute_output_shape(self, input_shape):
        return (input_shape[0], (input_shape[2] * self.k))

    def call(self, inputs):
        
        # swap last two dimensions since top_k will be applied along the last dimension
        shifted_input = tf.transpose(inputs, [0, 2, 1])
        
        # extract top_k, returns two tensors [values, indices]
        top_k = tf.nn.top_k(shifted_input, k=self.k, sorted=True, name=None)[0]
        
        # return flattened output
        return Flatten()(top_k)

In [19]:
from keras.layers import Input, Embedding, Dense
# from keras.layers import GlobalMaxPool1D, GlobalMaxPool2D, SpatialDropout1D, Dropout, BatchNormalization, Lambda
# from keras.layers import concatenate, Flatten, add, dot, PReLU, merge, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Add, Multiply
# from keras.layers import LSTM, Conv1D, GlobalMaxPool2D, Convolution2D, Conv2D, CuDNNGRU, CuDNNLSTM, GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.regularizers import l1, l2
from keras import optimizers, losses, activations
from keras import backend as K
from keras import optimizers
from keras.layers import *
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [20]:
def get_rnn_model():
    features_input = Input(shape=(features.shape[1],))

    
    inpword = Input(shape=(config.maxlen, ))
    emb_word = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False)(inpword)

    lDropout_titl = SpatialDropout1D(0.5)(emb_word)
    title_layer = Bidirectional(CuDNNLSTM(128, return_sequences=True))(lDropout_titl)

    title_layer = Bidirectional(CuDNNGRU(128, return_sequences=True))(title_layer)

    max_pool_til = GlobalMaxPooling1D()(title_layer)
    att = Attention(config.maxlen)(title_layer)
# AttentionWeightedAverage()

    all_views = concatenate([max_pool_til, att, features_input], axis=1)
    x = Dropout(0.2)(all_views)

    x = PReLU()(Dense(128)(x))

    x = Dense(20, activation="softmax")(x)
    model = Model(inputs=[inpword, features_input], outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    
    return model

In [21]:
one_y_train=np_utils.to_categorical(Y,num_classes=20)
one_y_val=np_utils.to_categorical(Y,num_classes=20)

In [22]:
from sklearn.model_selection import KFold
# pretrain=False
# if pretrain == True and len(X_test)<38:
#     X_test.append(test_x)
def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((102277,20)),np.zeros((102277,20))
#     train_pred, test_pred = np.zeros((train_p_s,1)),np.zeros((test_p_s,1))
    for train_index, test_index in kf.split(train):
        # x,y
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
#         curr_x1, curr_x2 = tr_word_pad[train_index], tr_article_pad[train_index]
        curr_x1 = tr_word_pad[train_index]
#         hold_out_x1,hold_out_x2 =tr_word_pad[test_index],tr_article_pad[test_index]
        hold_out_x1=tr_word_pad[test_index]
        curr_y, hold_out_y = one_y_train[train_index], one_y_train[test_index]
#         curr_x,curr_y = X[train_index],one_y_train[train_index]
#         hold_out_x,hold_out_y =X[test_index],one_y_val[test_index]
#         curr_x,curr_y = [np.array(x)[train_index] for x in X],one_y_train[train_index]
#         curr_other_x = train_x[train_index]
#         hold_out_x,hold_out_y = [np.array(x)[test_index] for x in X],one_y_train[test_index]
#         hold_out_other_x = train_x[test_index]
        kfold_X_features = features[train_index]
        kfold_X_valid_features = features[test_index]
        # curr_x ,curr_y= [x[train_idx] for x in X]
        # hold_out_x = [x[valid_idx] for x in X]
        # model
        config.batch_size = 32
        epochs = 15

        model = get_rnn_model()

        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, save_best_only=True,verbose=1, monitor='val_f1',  mode='max')
        early = EarlyStopping( monitor='val_f1',  mode='max', patience=2,)
        callbacks_list = [checkpoint,early]
        # train and pred
#         if pretrain == False:
        model.fit([curr_x1, kfold_X_features], curr_y, 
                  batch_size=config.batch_size, epochs=epochs, 
                    validation_data=([hold_out_x1,kfold_X_valid_features], hold_out_y), 
                  callbacks=callbacks_list)
#         else:
#             curr_x.append(curr_other_x)
#             hold_out_x.append(hold_out_other_x)
            
#             model.fit(X_test, test_y,
#             batch_size=config.batch_size, epochs=1,
#             validation_data=(hold_out_x, hold_out_y),
#             callbacks=callbacks_list)
            
            

#             model.fit(curr_x,  curr_y, 
#                       batch_size=config.batch_size, epochs=epochs, 
#                       validation_data=
#                       (hold_out_x, hold_out_y), 
#                       callbacks=callbacks_list)
        
        model.load_weights(file_path)
       
        y_test = model.predict([te_word_pad, test_features])
        test_pred += y_test
        hold_out_pred = model.predict([hold_out_x1,kfold_X_valid_features])
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    try:
        print('all eval',sqrt(mean_squared_error(Y,train_pred)))
    finally:
        return train_pred, test_pred


print('def done')

def done


In [23]:
%%time
import time
import pickle
model_time = time.time()
print("start")
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)
print (train_pred.shape,test_pred.shape)
print ("[{}] finished nn model".format((time.time()-model_time)/3600))

start
now is 1 fold
Train on 92049 samples, validate on 10228 samples
Epoch 1/15
92049/92049 [==============================] - 718s 8ms/step - loss: 1.4885 - f1: 0.4960 - val_loss: 1.0956 - val_f1: 0.6480

Epoch 00001: val_f1 improved from -inf to 0.64795, saving model to weights_base.best.h5
Epoch 2/15
 5696/92049 [>.............................] - ETA: 10:33 - loss: 1.1243 - f1: 0.6496

KeyboardInterrupt: 